# Assignment 7: Classification with Support Vector Machines

# Total: 11 Marks 
## Instructions

* Complete the assignment
* Once the notebook is complete, restart your kernel and rerun your cells
* Submit this notebook to owl by the deadline

## The Dataset

This dataset consists of 3921 e-mails to a single account, some of which are spam. These data represent incoming emails for the first three months of 2012 for an email account.

The table has 3921 (1252) observations on the following 21 variables.

* spam: Indicator for whether the email was spam.
* to_multiple: Indicator for whether the email was addressed to more than one recipient.
* from: Whether the message was listed as from anyone (this is usually set by default for regular outgoing email).
* cc: Indicator for whether anyone was CCed.
* sent_email: Indicator for whether the sender had been sent an email in the last 30 days.
* time: Time at which email was sent.
* image: The number of images attached.
* attach: The number of attached files.
* dollar: The number of times a dollar sign or the word “dollar” appeared in the email.
* winner: Indicates whether “winner” appeared in the email.
* inherit: The number of times “inherit” (or an extension, such as “inheritance”) appeared in the email.
* viagra: The number of times “viagra” appeared in the email.
* password: The number of times “password” appeared in the email.
* num_char: The number of characters in the email, in thousands.
* line_breaks: The number of line breaks in the email (does not count text wrapping).
* format: Indicates whether the email was written using HTML (e.g. may have included bolding or active links).
* re_subj: Whether the subject started with “Re:”, “RE:”, “re:”, or “rE:”
* exclaim_subj: Whether there was an exclamation point in the subject.
* urgent_subj: Whether the word “urgent” was in the email subject.
* exclaim_mess: The number of exclamation points in the email message.
* number: Factor variable saying whether there was no number, a small number (under 1 million), or a big number.

The data are from this R package: https://cran.r-project.org/web/packages/openintro/openintro.pdf

In [ ]:
# You may need these
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import recall_score, make_scorer, confusion_matrix
from sklearn.utils.multiclass import unique_labels

import matplotlib.pyplot as plt
%matplotlib inline

## Part A: 1 Mark

Read in the `email.txt` dataset.

In [ ]:
df = pd.read_csv('email.txt', delimiter='\t')

df = df.drop('time', axis = 'columns')
df = df.drop('number',axis = 'columns')
df = df.drop('winner',axis = 'columns')


df.head()

## Part B: 1 Mark

Split the data into train and test.  Hold out 50% of observations as the test set.  Pass `random_state=0` to `train_test_split` to ensure you get the same train and tests sets as the solution.

In [ ]:
X = df.drop('spam', axis='columns').astype('float').values
y = df.spam.values

Xtrain, Xtest, ytrain, ytest = train_test_split(X,
                                                y, 
                                                test_size=0.5, 
                                                random_state=0)

## Part C: 1 Mark

Create a pipeline for your support vector machine.  You can start with a `kernel="linear"` and `gamma="auto"`.

In [ ]:
pipe = Pipeline([
    ('SVC', SVC(kernel = 'linear', gamma = 'auto') )
])



## Part D: 3 Marks

Use your model to construct a confusion matrix by fitting and predicting on the training data (I've inlcluded a little helper function to make looking at the confusion matrix a little easier). Then answer the following using the confusion matrix (don't use sklearn's functions):

* What is your model's training accuracy?
* What is your model's training precision?
* What is your model's training recall?

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes=np.array([0.0,1.0]), normalize=False, title=None, cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig, ax = plt.subplots()
    ax.axis('equal')
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout();
    return ax

pipe.fit(Xtrain, ytrain)

prediction = pipe.predict(Xtrain)

plot_confusion_matrix(ytrain, prediction, classes = np.array([0, 1]));

matrix = confusion_matrix(ytrain, prediction)

diagsum = np.diag(matrix).sum()

accuracy = diagsum/matrix.sum()

precision = matrix[1,1]/np.sum(matrix[:,1])

recall = matrix[1,1]/np.sum(matrix[1,:])




In [ ]:
accuracy

In [ ]:
precision

In [ ]:
recall

## Part E:  1 Mark

Estimate your support vector machine's out of sample recall by using 5 fold cross validation.

In [ ]:
score = cross_val_score(pipe, Xtrain, ytrain, cv = 5, scoring=make_scorer(recall_score))

score


## Part F: 2 Marks

  Use sklearn's `GridSearchCV` to search over the kernel and gamma. Search over `kernel = ['rbf','sigmoid']` and `gamma = np.linspace(1e-5, 1e-2)`.  Use recall as your metric for scoring.
  

`GridSearchCV` is a way to cross validate your models for a variety of parameters.  Read more about `GridSearchCV` [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In [ ]:
svc_pipe = Pipeline([
    ('SVC', SVC() )
])




param_grid = {'SVC__kernel': ['rbf','sigmoid'],
              'SVC__gamma': np.linspace(1e-5, 1e-2)}

svc_gscv = GridSearchCV(svc_pipe, 
                          param_grid, 
                          cv = 5,
                          scoring=make_scorer(recall_score),
                          iid = True)

svc_gscv.fit(Xtrain, ytrain)

## Part G: 1 Mark

What was the cross validated recall for your regularized model?  If you called your model grid search `svc_gscv` you can access the best model's score by performing `svc_gscv.best_score_`.


In [ ]:
svc_gscv.best_score_


## Part H: 1 Mark

You can access the results of the cross validation `.cv_results_` method. If you called your estimator `svc_gcsv` then call `svc_gscv.cv_results_`.  This will return a dictionary.  You can turn it into a dataframe using `pandas.DataFrame` (this will make it easier to manipulate).  Learn more about `.cv_results_` [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

Plot how the mean test error changes as `gamma` changes.  Color the lines according to `kernel`. What do you see happening to the cross validated error as gamma increases?

In [ ]:
result = pd.DataFrame(svc_gscv.cv_results_)

fig, ax = plt.subplots(dpi=100)

sns.lineplot(data = cv_results)

